In [15]:
import yfinance as yf
import pandas as pd
import numpy as np
import hvplot.pandas
from empyrical import max_drawdown, alpha_beta
from empyrical import roll_max_drawdown
from empyrical import roll_up_capture, capture
import empyrical as em
from pathlib import Path

In [16]:
# Market Index Informations and pull their historical closing price since January 2018
start = "2018-01-01"
sp500 = yf.Ticker("^GSPC")
dows30 = yf.Ticker("^DJI")
nasdaq = yf.Ticker("^IXIC")
sptsx = yf.Ticker("^GSPTSE")
sp500_data = sp500.history(start= start, actions=False)
dows30_data = dows30.history(start = start)
nasdaq_data = nasdaq.history(start= start)
sptsx_data = sptsx.history(start= start)

In [17]:
sp500_data.drop(columns=["Open", "High", "Low", "Volume"], inplace=True)

In [18]:
# Fidelity funds Information and historical closing price since January 2018
fid_us_stocks = yf.Ticker("0P0001364K.TO")
fid_us_stocks_data = fid_us_stocks.history(start = start, actions=False)
fid_us_equity = yf.Ticker("0P000074PP.TO")
fid_us_equity_data = fid_us_equity.history(start = start, actions=False)
# NEI funds ...
nei_us_equity_rs = yf.Ticker("0P000070LH.TO")
nei_us_equity_rs_data = nei_us_equity_rs.history(start = start, actions=False)
# Invesco funds ...
aim_us_companies = yf.Ticker("0P000072U5.TO")
aim_us_companies_data = aim_us_companies.history(start = start, actions=False)
#CIBC funds ...
cibc_us_equity = yf.Ticker("0P000077O0.TO")
cibc_us_equity_data = cibc_us_equity.history(start = start, actions=False)
#RBC funds ...
rbc_us_equity = yf.Ticker("RBCUSEQUITYS.TO")
rbc_us_equity_data = rbc_us_equity.history(start = start, actions=False)
#Scotia funds ...
scotia_us_equity = yf.Ticker("0P000070D4.TO")
scotia_us_equity_data = scotia_us_equity.history(start = start, actions=False)
#BMO funds ...
bmo_us_equity = yf.Ticker("0P00014EDS.TO")
bmo_us_equity_data = bmo_us_equity.history(start = start, actions=False)
#Sunlife Funds
sunlife_us_growth = yf.Ticker("0P00015ILM.TO")
sunlife_us_growth_data = sunlife_us_growth.history(start = start, actions=False)
#Mawer Funds
mawer_us_equity = yf.Ticker("0P0000716B.TO")
mawer_us_equity_data = mawer_us_equity.history(start = start, actions=False)

In [19]:
# Drop extra columns - Open, High, Low, Volume

fid_us_stocks_data.drop(columns=["Open", "High", "Low", "Volume"], inplace=True)
fid_us_equity_data.drop(columns=["Open", "High", "Low", "Volume"], inplace=True)
nei_us_equity_rs_data.drop(columns=["Open", "High", "Low", "Volume"], inplace=True)
aim_us_companies_data.drop(columns=["Open", "High", "Low", "Volume"], inplace=True)
cibc_us_equity_data.drop(columns=["Open", "High", "Low", "Volume"], inplace=True)
rbc_us_equity_data.drop(columns=["Open", "High", "Low", "Volume"], inplace=True)
scotia_us_equity_data.drop(columns=["Open", "High", "Low", "Volume"], inplace=True)
bmo_us_equity_data.drop(columns=["Open", "High", "Low", "Volume"], inplace=True)
sunlife_us_growth_data.drop(columns=["Open", "High", "Low", "Volume"], inplace=True)
mawer_us_equity_data.drop(columns=["Open", "High", "Low", "Volume"], inplace=True)




In [20]:
mawer_us_equity_data.count()

Close    942
dtype: int64

In [21]:
# Combine into single df using concat function -excluding RBC for now - only 16 data points
daily_close_df = pd.concat([fid_us_stocks_data, fid_us_equity_data, nei_us_equity_rs_data, aim_us_companies_data, cibc_us_equity_data, scotia_us_equity_data, bmo_us_equity_data, sunlife_us_growth_data, mawer_us_equity_data, sp500_data], axis="columns", join="inner")

In [22]:
funds_columns = ["FID US Stocks", "FID US Equity", "NEI US Equity RS", "AIM US Companies", "CIBC US Equity", "Scotia US Equity", "BMO US Equity", "Sunlife US Growth", "Mawer US Equity", "S&P500"]
daily_close_df.columns=(funds_columns)


In [23]:
daily_returns = daily_close_df.pct_change()
daily_returns = daily_returns.dropna()
daily_returns

,FID US Stocks,FID US Equity,NEI US Equity RS,AIM US Companies,CIBC US Equity,Scotia US Equity,BMO US Equity,Sunlife US Growth,Mawer US Equity,S&P500
Date,,,,,,,,,,
2018-01-03,0.011615,0.007338,0.007122,0.010438,0.010296,0.008984,0.008046,0.013065,0.009637,0.006399
2018-01-04,0.007428,0.001941,0.002840,0.001109,-0.000562,0.001228,0.003287,0.002525,0.005464,0.004029
2018-01-05,0.015560,-0.000520,0.001880,-0.001247,0.003267,-0.000176,-0.000596,-0.000058,-0.002476,0.007034
2018-01-08,0.003022,0.005053,0.001745,0.001387,0.002338,0.002937,-0.000112,0.005075,0.004642,0.001662
2018-01-09,0.002904,0.004560,0.003358,0.010317,0.007281,0.004431,0.004599,0.006273,0.007178,0.001303
...,...,...,...,...,...,...,...,...,...,...
2021-09-27,-0.006965,-0.012471,-0.007017,-0.007729,-0.008829,-0.006200,-0.003258,-0.017305,-0.013631,-0.002776
2021-09-28,-0.032855,-0.013487,-0.015576,-0.027190,-0.018216,-0.017507,-0.016885,-0.025937,-0.016413,-0.020364
2021-09-29,-0.004471,0.011923,0.011171,0.000728,0.008069,0.007028,0.005404,0.002069,0.005819,0.001569


In [24]:
bench_mark_sp500 = daily_returns['S&P500'].reset_index()

In [25]:
bench_mark_sp500 = bench_mark_sp500.set_index('Date')

In [26]:
cumulative_returns = (1 + daily_returns).cumprod()
cumulative_returns

,FID US Stocks,FID US Equity,NEI US Equity RS,AIM US Companies,CIBC US Equity,Scotia US Equity,BMO US Equity,Sunlife US Growth,Mawer US Equity,S&P500
Date,,,,,,,,,,
2018-01-03,1.011615,1.007338,1.007122,1.010438,1.010296,1.008984,1.008046,1.013065,1.009637,1.006399
2018-01-04,1.019129,1.009294,1.009983,1.011559,1.009728,1.010223,1.011360,1.015623,1.015154,1.010453
2018-01-05,1.034987,1.008769,1.011881,1.010298,1.013027,1.010045,1.010757,1.015564,1.012640,1.017561
2018-01-08,1.038115,1.013866,1.013647,1.011699,1.015395,1.013011,1.010643,1.020718,1.017341,1.019252
2018-01-09,1.041130,1.018489,1.017051,1.022137,1.022788,1.017500,1.015291,1.027121,1.024643,1.020580
...,...,...,...,...,...,...,...,...,...,...
2021-09-27,1.992418,1.321897,1.564121,1.483993,1.222148,1.675752,1.634713,1.966668,1.793450,1.648154
2021-09-28,1.926957,1.304069,1.539758,1.443643,1.199886,1.646414,1.607111,1.915658,1.764014,1.614591
2021-09-29,1.918342,1.319618,1.556959,1.444694,1.209567,1.657985,1.615796,1.919621,1.774280,1.617124


In [27]:
# Comparison of all stocks using hvplot
cumulative_returns.hvplot.line(title = "Cumulative Return of Major Bank Funds and S&P500 since 2018", xlabel = "Date",ylabel = "Daily Return",width = 1000, height = 500)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [28]:
benchmark_return = daily_returns.loc[:,['S&P500']]

In [29]:
funds_columns

['FID US Stocks',
 'FID US Equity',
 'NEI US Equity RS',
 'AIM US Companies',
 'CIBC US Equity',
 'Scotia US Equity',
 'BMO US Equity',
 'Sunlife US Growth',
 'Mawer US Equity',
 'S&P500']

In [30]:
max_drawdown = em.max_drawdown(daily_returns)
df_max_drawdown = max_return.to_frame()
df_max_drawdown.columns =['Max Drawdown']

NameError: name 'max_return' is not defined

In [ ]:
annual_return = em.annual_return(daily_returns,period='daily',annualization= None)
df_annual_return = pd.DataFrame(annual_return, index=funds_columns,columns = ['Annual Fund'])

In [ ]:
annual_volatility = em.annual_volatility(daily_returns,period='daily',alpha=2.0,annualization= None)
df_annual_volatility = pd.DataFrame(annual_volatility,columns = ['Annual Volatility'])

In [ ]:
sharpe_ratio = em.sharpe_ratio(daily_returns,risk_free=0)
df_sharpe_ratio = pd.DataFrame(sharpe_ratio,index=funds_columns,columns=['Sharpe Ratio'])

In [ ]:
sortino_ratio = em.sortino_ratio(daily_returns,required_return = 0)
df_sortino_ratio = sortino_ratio.to_frame()
df_sortino_ratio.columns = ['Sortino Ratio']

In [ ]:
sortino_ratio

In [ ]:
downside_risk = em.downside_risk(daily_returns,required_return = 0)
df_downside_risk = pd.DataFrame(downside_risk,columns = ['Downside Risk']).reset_index()

In [ ]:
# Beta for each funds
beta_list=[]
for columns in daily_returns:
    beta = em.beta(daily_returns[columns],benchmark_return,risk_free = 0.0)
    beta_list.append(beta)
print(beta_list)

In [ ]:
df_beta =pd.DataFrame(beta_list,columns=['Beta'])

In [ ]:
# Alpha for each funds
alpha_list=[]
for columns in daily_returns:
    alpha = em.alpha(daily_returns[columns],benchmark_return,risk_free =0.0)
    alpha_list.append(alpha)
print(alpha_list)

In [ ]:
#Creating Alpha dataframe
df_alpha = pd.DataFrame(alpha_list,columns=['Alpha'])

In [31]:
combined = pd.concat([df_max_drawdown,df_downside_risk,df_sortino_ratio,df_annual_volatility,df_alpha,df_beta],axis = 1)
combined.set_index('index')

NameError: name 'df_max_drawdown' is not defined

In [32]:
#-------
file = Path('covid_usa_history.csv')
covid_cases = pd.read_csv(file).dropna()
covid_cases

,date,death,deathIncrease,inIcuCumulative,inIcuCurrently,hospitalizedIncrease,hospitalizedCurrently,hospitalizedCumulative,negative,negativeIncrease,onVentilatorCumulative,onVentilatorCurrently,positive,positiveIncrease,states,totalTestResults,totalTestResultsIncrease
0,2021-03-07,515151.0,842,45475.0,8134.0,726,40199.0,776361.0,74582825.0,131835,4281.0,2802.0,28756489.0,41835,56,363825123,1170059
1,2021-03-06,514309.0,1680,45453.0,8409.0,503,41401.0,775635.0,74450990.0,143835,4280.0,2811.0,28714654.0,60015,56,362655064,1430992
2,2021-03-05,512629.0,2221,45373.0,8634.0,2781,42541.0,775132.0,74307155.0,271917,4275.0,2889.0,28654639.0,68787,56,361224072,1744417
3,2021-03-04,510408.0,1743,45293.0,8970.0,1530,44172.0,772351.0,74035238.0,177957,4267.0,2973.0,28585852.0,65487,56,359479655,1590984
4,2021-03-03,508665.0,2449,45214.0,9359.0,2172,45462.0,770821.0,73857281.0,267001,4260.0,3094.0,28520365.0,66836,56,357888671,1406795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,2020-04-05,10618.0,1340,609.0,5811.0,1131,32180.0,13374.0,788594.0,33621,39.0,650.0,342104.0,25843,56,1940649,129412
337,2020-04-04,9278.0,1478,403.0,5500.0,1732,30456.0,12243.0,754973.0,143525,39.0,656.0,316261.0,33122,56,1811237,231115
338,2020-04-03,7800.0,1288,335.0,4928.0,1189,25723.0,10511.0,611448.0,54760,39.0,623.0,283139.0,31835,56,1580122,142407
339,2020-04-02,6512.0,1175,305.0,4513.0,1320,22995.0,9322.0,556688.0,51405,32.0,576.0,251304.0,28135,56,1437715,131146


In [33]:
covid_cases = covid_cases.set_index(pd.to_datetime(covid_cases['date'], infer_datetime_format=True))
covid_cases

,date,death,deathIncrease,inIcuCumulative,inIcuCurrently,hospitalizedIncrease,hospitalizedCurrently,hospitalizedCumulative,negative,negativeIncrease,onVentilatorCumulative,onVentilatorCurrently,positive,positiveIncrease,states,totalTestResults,totalTestResultsIncrease
date,,,,,,,,,,,,,,,,,
2021-03-07,2021-03-07,515151.0,842,45475.0,8134.0,726,40199.0,776361.0,74582825.0,131835,4281.0,2802.0,28756489.0,41835,56,363825123,1170059
2021-03-06,2021-03-06,514309.0,1680,45453.0,8409.0,503,41401.0,775635.0,74450990.0,143835,4280.0,2811.0,28714654.0,60015,56,362655064,1430992
2021-03-05,2021-03-05,512629.0,2221,45373.0,8634.0,2781,42541.0,775132.0,74307155.0,271917,4275.0,2889.0,28654639.0,68787,56,361224072,1744417
2021-03-04,2021-03-04,510408.0,1743,45293.0,8970.0,1530,44172.0,772351.0,74035238.0,177957,4267.0,2973.0,28585852.0,65487,56,359479655,1590984
2021-03-03,2021-03-03,508665.0,2449,45214.0,9359.0,2172,45462.0,770821.0,73857281.0,267001,4260.0,3094.0,28520365.0,66836,56,357888671,1406795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-05,2020-04-05,10618.0,1340,609.0,5811.0,1131,32180.0,13374.0,788594.0,33621,39.0,650.0,342104.0,25843,56,1940649,129412
2020-04-04,2020-04-04,9278.0,1478,403.0,5500.0,1732,30456.0,12243.0,754973.0,143525,39.0,656.0,316261.0,33122,56,1811237,231115
2020-04-03,2020-04-03,7800.0,1288,335.0,4928.0,1189,25723.0,10511.0,611448.0,54760,39.0,623.0,283139.0,31835,56,1580122,142407


In [34]:
covid_cases = covid_cases.drop(columns=['date', 
                                        'deathIncrease', 
                                        'inIcuCurrently', 
                                        'hospitalizedIncrease', 
                                        'hospitalizedCurrently', 
                                        'hospitalizedCumulative', 
                                        'negative', 
                                        'negativeIncrease', 
                                        'onVentilatorCumulative', 
                                        'onVentilatorCurrently', 
                                        'positiveIncrease', 
                                        'states', 
                                        'totalTestResults', 
                                        'totalTestResultsIncrease'])
covid_cases

,death,inIcuCumulative,positive
date,,,
2021-03-07,515151.0,45475.0,28756489.0
2021-03-06,514309.0,45453.0,28714654.0
2021-03-05,512629.0,45373.0,28654639.0
2021-03-04,510408.0,45293.0,28585852.0
2021-03-03,508665.0,45214.0,28520365.0
...,...,...,...
2020-04-05,10618.0,609.0,342104.0
2020-04-04,9278.0,403.0,316261.0
2020-04-03,7800.0,335.0,283139.0


In [35]:
covid_cases = covid_cases.sort_index(ascending=True)
covid_pct_change = covid_cases.pct_change().dropna()
covid_pct_change

,death,inIcuCumulative,positive
date,,,
2020-04-02,0.220161,0.191406,0.126070
2020-04-03,0.197789,0.098361,0.126679
2020-04-04,0.189487,0.202985,0.116981
2020-04-05,0.144428,0.511166,0.081714
2020-04-06,0.123752,0.088670,0.082606
...,...,...,...
2021-03-03,0.004838,0.002884,0.002349
2021-03-04,0.003427,0.001747,0.002296
2021-03-05,0.004351,0.001766,0.002406


In [36]:
covid_and_daily_joined = pd.concat([daily_returns, covid_pct_change], axis="columns", join="inner")
covid_and_daily_joined

,FID US Stocks,FID US Equity,NEI US Equity RS,AIM US Companies,CIBC US Equity,Scotia US Equity,BMO US Equity,Sunlife US Growth,Mawer US Equity,S&P500,death,inIcuCumulative,positive
2020-04-02,0.020584,0.020932,0.018635,0.024616,0.020936,0.019576,0.022389,0.014716,0.015708,0.022829,0.220161,0.191406,0.126070
2020-04-03,-0.016715,-0.023844,-0.017087,-0.017841,-0.018623,-0.017650,-0.017820,-0.017493,-0.011455,-0.015137,0.197789,0.098361,0.126679
2020-04-06,0.072038,0.054661,0.063553,0.073120,0.068544,0.067584,0.063355,0.076476,0.065283,0.070331,0.123752,0.088670,0.082606
2020-04-07,-0.011181,-0.016403,-0.011104,-0.013200,-0.014167,-0.009828,-0.004899,-0.017854,-0.013482,-0.001603,0.171136,0.113122,0.082125
2020-04-09,0.043988,0.061766,0.041854,0.045145,0.043299,0.048995,0.045438,0.028909,0.046315,0.049037,0.128592,0.064965,0.081048
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-01,0.040993,0.011474,0.018116,0.034032,0.017993,0.023337,0.013638,0.039153,0.026634,0.018928,0.002466,0.001091,0.001696
2021-03-02,-0.009125,-0.008602,-0.009390,-0.019060,-0.009149,-0.010819,-0.004962,-0.013295,-0.008928,-0.008081,0.003425,0.002847,0.001910
2021-03-03,-0.027399,-0.008814,-0.009629,-0.022316,-0.012226,-0.011407,-0.008763,-0.027045,-0.014337,-0.013066,0.004838,0.002884,0.002349
2021-03-04,-0.031221,-0.008026,-0.009594,-0.021613,-0.009101,-0.011911,-0.010083,-0.020603,-0.014289,-0.013417,0.003427,0.001747,0.002296


In [39]:
covid_correlation = covid_and_daily_joined.corr()
covid_correlation

,FID US Stocks,FID US Equity,NEI US Equity RS,AIM US Companies,CIBC US Equity,Scotia US Equity,BMO US Equity,Sunlife US Growth,Mawer US Equity,S&P500,death,inIcuCumulative,positive
FID US Stocks,1.000000,0.652716,0.845368,0.813004,0.867271,0.869074,0.846442,0.920412,0.851642,0.891959,0.173599,0.143011,0.180339
FID US Equity,0.652716,1.000000,0.841522,0.687837,0.828303,0.831521,0.788238,0.612323,0.818003,0.737214,0.181320,0.164334,0.194215
NEI US Equity RS,0.845368,0.841522,1.000000,0.803235,0.971874,0.964982,0.929595,0.859735,0.933412,0.873683,0.193725,0.174563,0.205987
AIM US Companies,0.813004,0.687837,0.803235,1.000000,0.823352,0.834743,0.815275,0.772348,0.804123,0.795521,0.177074,0.169142,0.179115
CIBC US Equity,0.867271,0.828303,0.971874,0.823352,1.000000,0.982579,0.960688,0.857787,0.945165,0.915658,0.196768,0.183319,0.207746
Scotia US Equity,0.869074,0.831521,0.964982,0.834743,0.982579,1.000000,0.974137,0.838080,0.944347,0.943707,0.202343,0.180062,0.210785
BMO US Equity,0.846442,0.788238,0.929595,0.815275,0.960688,0.974137,1.000000,0.795260,0.917013,0.943640,0.211671,0.210286,0.216925
Sunlife US Growth,0.920412,0.612323,0.859735,0.772348,0.857787,0.838080,0.795260,1.000000,0.851304,0.783086,0.149710,0.117835,0.160688
Mawer US Equity,0.851642,0.818003,0.933412,0.804123,0.945165,0.944347,0.917013,0.851304,1.000000,0.894122,0.188501,0.180292,0.206776
S&P500,0.891959,0.737214,0.873683,0.795521,0.915658,0.943707,0.943640,0.783086,0.894122,1.000000,0.202106,0.200189,0.203974


In [40]:
covid_correlation.style.background_gradient(cmap="summer")

,FID US Stocks,FID US Equity,NEI US Equity RS,AIM US Companies,CIBC US Equity,Scotia US Equity,BMO US Equity,Sunlife US Growth,Mawer US Equity,S&P500,death,inIcuCumulative,positive
FID US Stocks,1.000000,0.652716,0.845368,0.813004,0.867271,0.869074,0.846442,0.920412,0.851642,0.891959,0.173599,0.143011,0.180339
FID US Equity,0.652716,1.000000,0.841522,0.687837,0.828303,0.831521,0.788238,0.612323,0.818003,0.737214,0.181320,0.164334,0.194215
NEI US Equity RS,0.845368,0.841522,1.000000,0.803235,0.971874,0.964982,0.929595,0.859735,0.933412,0.873683,0.193725,0.174563,0.205987
AIM US Companies,0.813004,0.687837,0.803235,1.000000,0.823352,0.834743,0.815275,0.772348,0.804123,0.795521,0.177074,0.169142,0.179115
CIBC US Equity,0.867271,0.828303,0.971874,0.823352,1.000000,0.982579,0.960688,0.857787,0.945165,0.915658,0.196768,0.183319,0.207746
Scotia US Equity,0.869074,0.831521,0.964982,0.834743,0.982579,1.000000,0.974137,0.838080,0.944347,0.943707,0.202343,0.180062,0.210785
BMO US Equity,0.846442,0.788238,0.929595,0.815275,0.960688,0.974137,1.000000,0.795260,0.917013,0.943640,0.211671,0.210286,0.216925
Sunlife US Growth,0.920412,0.612323,0.859735,0.772348,0.857787,0.838080,0.795260,1.000000,0.851304,0.783086,0.149710,0.117835,0.160688
Mawer US Equity,0.851642,0.818003,0.933412,0.804123,0.945165,0.944347,0.917013,0.851304,1.000000,0.894122,0.188501,0.180292,0.206776
S&P500,0.891959,0.737214,0.873683,0.795521,0.915658,0.943707,0.943640,0.783086,0.894122,1.000000,0.202106,0.200189,0.203974
